In [1]:
import pandas as pd
import requests
from datetime import timedelta, date
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows',200)

In [ ]:
# start = date(2017, 6, 1)
# end = date(2018, 6, 1)


# df = pd.DataFrame()

# for sdate in range((end - start).days):
#     fdate = (start + timedelta(sdate)).strftime('%y%m%d')
#     url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_" + fdate + ".txt"
#     try:
#         data = pd.read_csv(url)
#         data['file'] = fdate
#         df = df.append(data)
#     except:
#         pass

In [ ]:
# df['date'] = pd.to_datetime(df.DATE)

In [ ]:
# df['turnid'] = df['STATION'] + '/' + df['SCP'] + '/' + df['C/A']

In [ ]:
# # Remove readings at unusual times (that don't land on an even hour)
# df = df[df.TIME.str.contains(':00:00', regex=False)]


# df = df.sort_values(['STATION','turnid','TIME'])


# df = df.rename(columns={ df.columns[10]: "EXITS" })

In [ ]:
# pickle.dump(df, open('turnstiledata','wb'))

In [9]:
df = pickle.load(open('turnstiledata','rb'))

In [10]:
conditions = [  df.TIME.isin(['23:00:00','00:00:00','01:00:00','02:00:00']),
                df.TIME.isin(['03:00:00','04:00:00','05:00:00','06:00:00']),
                df.TIME.isin(['07:00:00','08:00:00','09:00:00','10:00:00']),
                df.TIME.isin(['11:00:00','12:00:00','13:00:00','14:00:00']),
                df.TIME.isin(['15:00:00','16:00:00','17:00:00','18:00:00']),
                df.TIME.isin(['19:00:00','20:00:00','21:00:00','22:00:00'])]

choices = ['00:00','04:00','08:00','12:00','16:00','20:00']

df['TIME'] = np.select(conditions, choices, default=df.TIME)

In [11]:
df = df.groupby(['STATION','turnid','date','TIME'], as_index=False).sum()

df['dayofweek'] = df.date.dt.day_name()


In [12]:
df['entries'] = df.ENTRIES.shift(-1) - df.ENTRIES
df['exits'] = df.EXITS.shift(-1) - df.EXITS
df['passages'] = df.entries + df.exits


In [13]:

# Delete last row for each turnstile, for which we can't calculate the net # entries
df = df.groupby(['turnid'], as_index=False).apply(lambda x: x.iloc[:-1])
#df['weekend/weekday'] = np.where(df['dayofweek'].isin( ['Saturday','Sunday']), 'Weekend','Weekday')

In [14]:
df = df.groupby(['STATION','date','dayofweek','TIME'], as_index=False).sum()


In [15]:
df_median = df.groupby(['STATION','dayofweek','TIME'], as_index=False).median()
df_median

,STATION,dayofweek,TIME,ENTRIES,EXITS,entries,exits,passages
0,1 AV,Friday,00:00,3.039575e+09,1.638959e+09,704.0,489.0,1206.5
1,1 AV,Friday,04:00,3.039711e+09,1.639100e+09,1267.0,2860.5,4137.5
2,1 AV,Friday,08:00,3.039712e+09,1.639103e+09,4825.0,5798.0,10539.0
3,1 AV,Friday,12:00,3.039717e+09,1.639107e+09,4549.5,4320.0,8964.0
4,1 AV,Friday,16:00,3.039722e+09,1.639111e+09,7010.5,7082.5,13938.5
5,1 AV,Friday,20:00,3.039660e+09,1.639048e+09,4863.0,4532.0,9526.0
6,1 AV,Monday,00:00,3.039626e+09,1.639013e+09,377.0,304.0,703.5
7,1 AV,Monday,04:00,3.039626e+09,1.639013e+09,1336.0,2804.0,4104.5
8,1 AV,Monday,08:00,3.039628e+09,1.639016e+09,4775.0,5805.0,10613.5
9,1 AV,Monday,12:00,3.039633e+09,1.639021e+09,3650.0,3747.0,7352.0


In [ ]:
df_count = df.groupby(['STATION','dayofweek','TIME'], as_index=False). count()

In [ ]:
df_median['count'] = df_count.iloc[:,4]

In [ ]:
df_median = df_median.drop(columns =['ENTRIES','EXITS'])


In [ ]:
df_median.head()

In [ ]:
plt.hist(df_median["count"], bins=10);

In [ ]:
df_median.sort_values(by= 'passages', ascending = False)

In [ ]:
df_median[df_median['count']>4].sort_values(by='passages', ascending = False)

In [ ]:
df_median[df_median['count']<=4].sort_values(by='passages', ascending = False)

In [ ]:
plt.hist(x = penn_station_passages);

In [ ]:
penn_station_passages=df_median[df_median['STATION']=='34 ST-PENN STA'].passages

In [ ]:
df_median['TIME'].value_counts()